# Problema clasificación con RNA y Keras

In [65]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

In [66]:
from numpy.random import seed
seed(1)

In [67]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [69]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Significado de las columnas:

 0. Number of times pregnant
 1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 2. Diastolic blood pressure (mm Hg)
 3. Triceps skin fold thickness (mm)
 4. 2-Hour serum insulin (mu U/ml)
 5. Body mass index (weight in kg/(height in m)^2)
 6. Diabetes pedigree function
 7. Age (years)
 8. **Variable clase (Tiene diabetes)**  ———> VARIABLE OBJETIVO

In [70]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

### Definimos el modelo Keras

In [72]:
# Para obtener todos el mismo resultado debemos añadir una semilla
tf.random.set_seed(1) 
np.random.seed(1)
random.seed(1)
seed(1)

In [73]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, activation='relu'))
# model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


### Compilamos el modelo

In [74]:
# La compilación usa (internamente) librerías numéricas muy eficientes como TensorFlow además de comprobar si tenemos GPU 
# o sólo CPU
# Es necesario definir la función de pérdida que vamos a minimizar (optimizar).  Para este caso minimizaremos 
# Binary Cross Entropy puesto que funciona bien para problemas binarios de clasificación.
# Como métrica (al ser clasificación) usaremos la precisión.
# Como optimizador, usaremos el algoritmo "adam" ya que ofrece buenos resultados en un amplio abanico de problemas y 
# además de manera rápida

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              #loss="binary_crossentropy", por que es una clasificación binaria
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, # usar cuando valores de predicción sean [0,1]
                                                      label_smoothing=0.0,
                                                      axis=-1,
                                                      reduction="auto", # ME DA ERROR si le pongo AUTO
                                                      name="binary_crossentropy"), 
              metrics=['accuracy']) # La métrica eleginda


ValueError: Invalid value for argument `reduction`. Expected one of {'none', 'sum_over_batch_size', None, 'sum'}. Received: reduction=auto

In [75]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

### Entrenamos el modelo

In [76]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5780 - loss: 2.3916
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6194 - loss: 1.0147
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6078 - loss: 0.9236
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6620 - loss: 0.8530
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6748 - loss: 0.8017
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6839 - loss: 0.7747
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6898 - loss: 0.7419
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6958 - loss: 0.7211
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6929 - loss: 0.7094
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6955 - loss: 0.6965
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7057 - loss: 0.6848
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

### Evaluamos el modelo

In [77]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.7314 - loss: 0.5314
Accuracy: 75.98


In [78]:
# No es un mal resultado, tenemos una precisión de más del 75%

### Predicciones

In [79]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
    

In [80]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[123,  20],
       [ 48,  40]])

## Ejercicios

1.Configura la red neuronal para que trabaje con 3 capas. 
- La primera con 15 neuronas y función de activación sigmoide. 
- La segunda con 10 neuronas y función de activación sigmoide.
- La tercera con 8 neuronas y función de activación ReLU.
- Una capa de salida con 1 neurona y función de activación sigmoide.

In [81]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

In [82]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [84]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [85]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [86]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

In [87]:
tf.random.set_seed(1) 

In [88]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [89]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

Entrenamos modelo

In [90]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6572 - loss: 0.6479
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6420
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6366
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6336
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6315
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6571 - loss: 0.6305
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6284
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6572 - loss: 0.6267
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6602 - loss: 0.6256
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6599 - loss: 0.6230
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6602 - loss: 0.6219
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy:

#### Evaluamos modelo

In [91]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.7344 - loss: 0.5218
Accuracy: 73.93


#### Predicciones

In [92]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
    

In [93]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[114,  29],
       [ 52,  36]])

2.Configura la red neuronal para que trabaje con las 3 mismas capas del ejemplo inicial, pero esta vez usa como función de activación de la capa de salida 'softmax'

In [106]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [107]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [108]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3428 - loss: 0.7808
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6706
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6475
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.3428 - loss: 0.6385
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3432 - loss: 0.6343
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3424 - loss: 0.6317
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6300
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6283
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6262
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3432 - loss: 0.6240
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3415 - loss: 0.6201
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3432 - 

In [97]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - accuracy: 0.3470 - loss: 0.5721
Accuracy: 33.52


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [98]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [99]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0, 143],
       [  0,  88]])

### **3.En compile, cambia la configuración del optimizer, de manera que en vez de Adam, usemos esta vez SGD:**
optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD')

In [125]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [120]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="SGD" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [121]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 955us/step - accuracy: 0.6572 - loss: 0.6338
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.6572 - loss: 0.6336
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.6572 - loss: 0.6334
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.6572 - loss: 0.6331
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.6572 - loss: 0.6329
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.6572 - loss: 0.6326
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6568 - loss: 0.6328
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.6584 - loss: 0.6311
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6317
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.6568 - loss: 0.6318
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6575 - loss: 0.6307
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

In [122]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6673 - loss: 0.5932
Accuracy: 67.04


In [123]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [124]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[133,  10],
       [ 76,  12]])

3.1.Cambia ahora la función de activación de la capa de salida a sigmoide

In [ ]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

4.En compile, cambia la configuración del loss para usar como función de pérdida CategoricalCrossentropy.

tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy"),


5.Prueba diferentes configuraciones a ver si consigues mejorar el resultado inicial.